In [1]:
import keras
import keras.backend as K
import kapre
import pandas
import seaborn
import pandas as pd
import scipy.io.wavfile as wav
seaborn.set_style("whitegrid")

%pylab inline
# https://github.com/keunwoochoi/kapre

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
ex_df = pd.read_pickle("data/ex_df.pkl")
ex_df.sample(10)

,fn,raw_label,state,label,label_weight,label_i,raw_label_i
222096,data/test/audio/clip_764ae2ee6.wav,NaN,submission,NaN,NaN,NaN,NaN
36969,data/train/audio/down/aa753bb9_nohash_0.wav,down,val,down,2.368270,3.0,3.0
205958,data/test/audio/clip_ada6c5e5a.wav,NaN,submission,NaN,NaN,NaN,NaN
118554,data/test/audio/clip_d69a642dc.wav,NaN,submission,NaN,NaN,NaN,NaN
221240,data/test/audio/clip_02c5ea061.wav,NaN,submission,NaN,NaN,NaN,NaN
76266,data/test/audio/clip_30bdab2f2.wav,NaN,submission,NaN,NaN,NaN,NaN
57453,data/train/audio/marvin/325a0c39_nohash_2.wav,marvin,train,unknown,0.136133,11.0,30.0
82130,data/test/audio/clip_242dd784d.wav,NaN,submission,NaN,NaN,NaN,NaN
198554,data/test/audio/clip_e4db3be01.wav,NaN,submission,NaN,NaN,NaN,NaN
118556,data/test/audio/clip_a30801bd5.wav,NaN,submission,NaN,NaN,NaN,NaN


In [3]:
ex_df.state.value_counts()

submission    158538
train          52888
test            7095
val             7058
bg                 6
Name: state, dtype: int64

In [4]:
bg_waves = [wav.read(fn)[1] for fn in ex_df[ex_df.state == "bg"].fn]
maxlen = 16000

def center_wave(wav_fn, vol_range=.1, displacement=.1, p_transform=.5):
    
    if wav_fn == "silence":
        bg_wave = random.choice(bg_waves)
        bg_start = np.random.randint(len(bg_wave)-maxlen)
        wave = bg_wave[bg_start:bg_start+maxlen]

    else:
        wave = wav.read(wav_fn)[1]
        
        left_pad = (maxlen - wave.shape[0]) // 2
        right_pad = maxlen - wave.shape[0] - left_pad
        
        wave = np.pad(
            wave, (left_pad, right_pad), 'constant', constant_values=0)
       
        if vol_range > 0:
            if np.random.rand() < p_transform:
                
                bg_vol = vol_range * np.random.rand(1)
                wave = (1-bg_vol)*wave + bg_vol*center_wave("silence", maxlen)
                wave = np.clip(wave, -1, 1)
        
        if displacement > 0:
            if np.random.rand() < p_transform:
                displ = int(displacement*(2*np.random.rand()-1)*maxlen)
                wave = np.roll(wave,displ)

    return wave

/usr/local/lib/python3.5/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [5]:
def ex_generator(batch_size=32,
                 shuffle=True,
                 state="train",
                 vol_range=0,
                 displacement=0,
                 p_transform=0):

    maxlen = 16000

    epoch_df = ex_df[ex_df.state == state]
    num_ex = len(epoch_df)
    indices = np.arange(num_ex)

    # epoch loop runs
    while True:

        # shuffle anew every epoch
        if shuffle:
            epoch_df = epoch_df.sample(frac=1)

        # batch loop
        for i in np.arange(0, num_ex, batch_size):

            x = []

            batch_df = epoch_df.iloc[i:i + batch_size:, :]

            # example loop
            for fn in batch_df.fn:

                #get the processed file
                item = center_wave(fn, vol_range=vol_range, displacement=displacement, p_transform=p_transform)

                x.append(item.reshape((1,1,maxlen)))

            x = np.concatenate(x,)
            y = batch_df.raw_label_i.values[:, np.newaxis]
            cw = batch_df.label_weight.values

            yield x, y, cw

In [10]:
dropout_prob = .5
init_stddev = 0.01
num_cat = 12

batch_size = 32

kernel_size = (3, 3)

cnn_layers = [
    keras.layers.InputLayer(input_shape=(1, 16000)),
    
    kapre.time_frequency.Melspectrogram(
        sr=16000,
        n_mels=125,
        n_dft=256,
        return_decibel_melgram=True),
    
#     kapre.time_frequency.Spectrogram(n_dft=512, n_hop=256, 
#           return_decibel_spectrogram=False, power_spectrogram=2.0, 
#           trainable_kernel=False),

    keras.layers.Conv2D(
        64,
        kernel_size=kernel_size,
        padding="same",
        activation="relu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    keras.layers.Dropout(dropout_prob),
    
    keras.layers.Conv2D(
        64,
        kernel_size=kernel_size,
        padding="same",
        activation="relu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    keras.layers.Dropout(.9 * dropout_prob),
    
    keras.layers.Conv2D(
        64,
        kernel_size=kernel_size,
        padding="same",
        activation="relu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    keras.layers.Dropout(.8 * dropout_prob),
    
#     keras.layers.Conv2D(
#         64,
#         kernel_size=kernel_size,
#         padding="same",
#         activation="relu",
#         kernel_initializer=keras.initializers.TruncatedNormal(
#             stddev=init_stddev)),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
#     keras.layers.Dropout(.7 * dropout_prob),
    
#     keras.layers.Conv2D(
#         64,
#         kernel_size=kernel_size,
#         padding="same",
#         activation="relu",
#         kernel_initializer=keras.initializers.TruncatedNormal(
#             stddev=init_stddev)),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
#     keras.layers.Dropout(.6 * dropout_prob),
    
    keras.layers.Flatten(),

    # Hidden Layer 1
    keras.layers.Dense(
        256,
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.Dropout(dropout_prob),

    # Classification Layer
    keras.layers.Dense(
        int(ex_df.raw_label_i.max()+1),
        activation="softmax",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
]
cnn_model = keras.Sequential(cnn_layers)
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 16000)          0         
_________________________________________________________________
melspectrogram_1 (Melspectro (None, 125, 125, 1)       82173     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 125, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        36928     
__________

In [35]:
def acc_12(y_true, y_pred):
    """ given y vectors based on 32 categories, returns accuracy based on first 12"""
    y_pred = K.cast(K.argmax(y_pred, axis=-1), K.floatx())
    y_true = K.max(y_true, axis=-1)
    y_true = K.minimum(y_true, 11.)
    y_pred = K.minimum(y_pred, 11.)
    return K.mean(K.equal(y_true, y_pred))


def comb_loss(y_true_32, y_pred_32):
    """ given y vectors based on 32 categories, returns accuracy based on first 12"""
    mask = K.greater_equal(K.reshape(K.arange(0, 32), (-1, 32)), 11)
    masked_y = K.cast(mask, "float32")*y_pred_32
    top_21 = K.sum(masked_y, axis=1, keepdims=True)
    y_pred_12 = K.concatenate((y_pred_32[:, 0:11], top_21), axis=1)
    y_true_12 = K.minimum(y_true_32, 11.)

    return (K.sparse_categorical_crossentropy(y_true_12, y_pred_12) +
            K.sparse_categorical_crossentropy(y_true_32, y_pred_32)) / 2

In [36]:
y_pred_32 = K.ones((64,32))*.1 

In [37]:
mask = K.greater_equal(K.reshape(K.arange(0, 32), (-1, 32)), 11)
K.cast(mask, "float32")*y_pred_32

<tf.Tensor 'mul_11:0' shape=(64, 32) dtype=float32>

In [38]:
batch_size = 32
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='logs/tb',
        histogram_freq=0,
        batch_size=batch_size,
        write_graph=False,
        write_grads=False,
        write_images=False,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='logs/cnn/mel.weights.{epoch:02d}-{val_loss:.3f}.hdf5',
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        mode='auto',
        period=1),
    keras.callbacks.CSVLogger('logs/cnn/spec-training.log'),
    keras.callbacks.EarlyStopping(patience=6, verbose=1),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.2, patience=1, verbose=1, min_lr=1e-8)
]

cnn_model = keras.Sequential(cnn_layers)
cnn_model.compile(
    loss=comb_loss,
    optimizer='nadam',
    metrics=['accuracy', acc_12])

In [39]:
cnn_model.fit_generator(
    generator=ex_generator(
        batch_size=batch_size,
        shuffle=True,
        state="train",
        vol_range=0,
        displacement=0,
        p_transform=0),
    steps_per_epoch=sum(ex_df.state == "train") / batch_size,
    epochs=100,
    verbose=1,
    callbacks=callbacks,
    validation_data=ex_generator(
        batch_size=batch_size,
        shuffle=True,
        state="train",
        vol_range=0,
        displacement=0,
        p_transform=0),
    validation_steps=sum(ex_df.state == "train") / batch_size,
    shuffle=False,
)

Epoch 1/100
1653/1652 [==============================] - 311s 188ms/step - loss: 2.4787 - acc: 0.0778 - acc_12: 0.0800 - val_loss: 2.1041 - val_acc: 0.0782 - val_acc_12: 0.1218
Epoch 2/100
 369/1652 [=====>........................] - ETA: 3:04 - loss: 2.3518 - acc: 0.0690 - acc_12: 0.0927

KeyboardInterrupt: 